# EM 算法

作者： [王塑](https://github.com/james016)

[参考文本](PDF/13.EM算法.pdf) 以及参考书 统计学习方法（李航）

## 目录

* [最大似然估计(MLE)数学原理](?#最大似然估计数学原理)
* [EM 算法数学原理](?#EM-算法数学原理)
* [GMM基本实现](?#GMM基本实现)
* [K-means 关系](?#K-means-关系)
* [EM/MLE 与 supervised 的比较](?#EM/MLE-与-supervised-的比较)
* [EM 算法基于 F 函数的表示](#EM-算法基于-F-函数的表示)


## 最大似然估计数学原理

考虑高斯概率密度分布
$$
f\left(x_{i}|\mu,\sigma^{2}\right)=\frac{1}{\sqrt{2\pi}\sigma}e^{-\left(x_{i}-\mu_{k}\right)^{2}/2\sigma_{k}^{2}},
$$
 当存在 $x_{1},x_{2},\dots,x_{i}$ 时，每个数都是独立函数，则概率为
$$
L\left(\mathbf{x}\right)=\prod_{i}f\left(x_{i}|\mu,\sigma^{2}\right),
$$
当 $\ln L\left(x\right)$ 对 $\mu$, $\sigma^{2}$ 取极值时，即可得到 $\mu$
和 $\sigma^{2}$ 的取值，表达式为
$$
\mu  =  \frac{1}{n}\sum_{i}\left(x_{i}\right)x_{i},
$$
$$
\sigma^{2} = \frac{1}{n}\sum_{i}\left(x_{i}-\mu\right)^{2}.
$$

## EM 算法数学原理

* 目标：求解含隐变量（未观测数据）的极大似然估计。以 $Y$ 为观测数据， $Z$ 为未观测数据。作极大似然函数 $P(Y|\theta)=\sum_Z P(Z|\theta)P(Y|Z,\theta)$ 估计。
* 解决问题：含隐变量的极大似然估计中不存在解析解。因为 $L(\theta)=\ln P(Y|\theta)$ 存在求和，无法分解为 $y_i$ 结果的简单求和形式。
* 实现方法：
    * 已知：观测变量数据 $Y$，隐变量数据 $Z$，联合分布 $P(Y,Z|\theta)$，条件分布 $P(Z|Y,\theta)$
    * 求解：模型参数 $\theta$
    * E步（期望）：计算 $P(Z|Y,\theta^{(i)})$ ，用以计算 $$
    Q(\theta,\theta^{(i)}) = \sum_Z\log P(Y,Z|\theta) P(Z|Y,\theta^{(i)}),
    $$ $Q(\theta,\theta^{(i)})$ 的特点是**求和中只有 $\log P(Y,Z|\theta)$ 关于 $\theta$ 函数，可以解析求解**
    * M步（极大）：$$
    \theta^{(i+1)}= \arg \max_{\theta} Q(\theta,\theta^{(i)})
    $$
    * 循环 E-M 步直至 $||\theta^{(i+1)}-\theta^{(i)}||<\epsilon$
    
    
实现逻辑
$
L\left(Y|\theta\right)=\ln\sum_{Z}P\left(Z|\theta\right)P\left(Y|Z,\theta\right)\underbrace{\longrightarrow}_{\begin{array}{c}
\text{No analysis results}\\
\text{Find approximation}\\
\text{Jensen inequality}
\end{array}}\text{approximate}\arg\max_{\theta}B\left(\theta,\theta^{\left(i\right)}\right)=\arg\max_{\theta}Q\left(\theta,\theta^{\left(i\right)}\right)
$

## GMM基本实现

以双正态分布为例

描述系统状态存在三类参数，均值、方差、每类所占的比重 $N\left(\mu_{1},\sigma_{1}^{2}\right)$,$N\left(\mu_{2},\sigma_{2}^{2}\right)$,
$\pi_{1},$$\pi_{2}$，EM 算法的目标是求解得到上述参数与数据最吻合的参数值。

执行 EM 算法分为两步：

1. 计算每个样本在每个分类的几率 
$$
\gamma_{k}\left(x_{i}\right)=\frac{\pi_{k}N\left(x_{i}|\mu_{k},\sigma_{k}^{2}\right)}{\sum_{k'}\pi_{k'}N\left(x_{i}|\mu_{k'},\sigma_{k'}^{2}\right)},
$$
其中$N\left(x_{i}|\mu_{k},\sigma_{k}^{2}\right)=\exp\left[-\left(x_{i}-\mu_{k}\right)^{2}/2\sigma_{k}^{2}\right]/\sqrt{2\pi}\sigma$
2. 通过 $\gamma_{k}\left(x_{i}\right)$ 重新计算 $\mu_{k}$, $\sigma_{k}^{2}$,
$\pi_{k}$
$$
n_{k}  = \sum_{i}\gamma_{k}\left(x_{i}\right),\quad\pi_{k}=\frac{n_{k}}{N},
$$
$$
\mu_{k} = \frac{1}{n_{k}}\sum_{i}\gamma_{k}\left(x_{i}\right)x_{i},
$$
$$
\sigma_{k}^{2} = \frac{1}{n_{k}}\sum_{i}\gamma_{k}\left(x_{i}\right)\left(x_{i}-\mu_{k}\right)^{2}.
$$

## K-means 关系

[注释，用 Gauss 方法解释]()<!---
在上面算法中设置 均值、方差、每类所占的比重 $N\left(\mu_{1},\sigma_{1}^{2}\right)$,$N\left(\mu_{2},\sigma_{2}^{2}\right)$,
$\pi_{1},$$\pi_{2}$，存在 $\sigma_{1}^{2}=\sigma_{2}^{2}=1$， $\pi_{1}=\pi_{2}=1$，同时计算
$\gamma_{k}\left(x_{i}\right)$ 的算法中，修改为 
$$
\gamma_{k}'\left(x_{i}\right) & = & \frac{\pi_{k}N\left(x_{i}|\mu_{k},\sigma_{k}^{2}\right)}{\sum_{k'}\pi_{k'}N\left(x_{i}|\mu_{k'},\sigma_{k'}^{2}\right)},\\
\gamma_{k}\left(x_{i}\right) & = & \begin{cases}
1 & \gamma_{k}'\left(x_{i}\right)=\max_{k}\gamma_{k}'\left(x_{i}\right)\\
0 & \mathrm{otherwise}
\end{cases}.
$$
-->



用 EM 算法标准表示可以修正为

* 观测数据 Y：各个已知点的位置
* 隐变量 Z：每个样本都有属于自己的分类
* 模型参数 $\theta$：K个分类点位置的中心
* 联合分布：
$$
p\left(Y,Z|\theta\right)dx^{d}=p\left(Y|\theta\right)dx^{d}P\left(Z|Y,\theta\right)=\frac{1}{S}dx^{d}P\left(Z|Y,\theta\right)
$$
其中
$$
P\left(Z|Y,\theta\right)=\begin{cases}
1 & \arg\min_{k}||Y-\theta_{k}||=Z\\
0 & otherwise
\end{cases},
$$
这里 $p\left(Y|\theta\right)$ 为常数与 $\theta$ 无关
* 条件分布：
$$
P\left(Z|Y,\theta\right)=\frac{P\left(Y,Z|\theta\right)}{\sum_{Z}P\left(Y,Z|\theta\right)}=\begin{cases}
1 & \arg\min_{k}||Y-\theta_{k}||=Z\\
0 & otherwise
\end{cases}.
$$

这里，对 $Q\left(\theta,\theta^{\left(i\right)}\right)$ 求极大的过程，<font color=red>应该</font>是各个位置求平均。


## EM/MLE 与 supervised 的比较

| 项目 | EM/MLE |  | supervised |
|:---| :--- | :----: | ----: |
| 核心 | $\max L(\theta )$ | $\longleftrightarrow$ | $L(\theta)$ SVM/softmax/L2 |
| 增加样本 | $\log P(y\vert \theta)$ (MLE)| $\longleftrightarrow$ | $(f(x)-y)^2$ （L2）|
| 目标 | 用少量参数描述海量样本 | $\longleftrightarrow$    | 新 $x \to y$   |
| | （可理解model$\to$隐参数,不可理解$\to$几率分布） |  | (不可理解$\to$取值预测分布) |

问题：
* 当我们获得一个不可理解的概率分布，可以描述我们的数据，我们能用来干什么？
    * 从已知部分参数，预测另一部分参数可能出现的概率（条件概率）$->$中文分词
    * 对分类问题，联合出现概率与预测结果两个作用，在预测模型中压低高概率样本的权重，用以减少不平衡数据的问题
* 可理解的概率分布（隐参数模型）的用途？
    * 用隐参数为样本分类，做关联性推测（新闻关联、人群关联）

## EM 算法基于 F 函数的表示

F 函数定义为
$$
F\left(\tilde{P},\theta\right)=\underbrace{\sum_{Z}\tilde{P}\left(Z\right)\ln P\left(Y,Z|\theta\right)}_{\textrm{like }Q\left(\theta,\theta^{(i)}\right),\,\theta^{(i)}\to\tilde{P}}-\underbrace{\sum_{Z}\tilde{P}\left(Z\right)\ln\tilde{P}\left(Z\right)}_{\text{Entropy of }\tilde{P}}
$$


* 目标：用 $F\left(\tilde{P},\theta\right)$ 的极大值得到 极大似然估计$L\left(\theta\right)$的极大值
* 方法：
    * 引理1$\to\arg\max_{\tilde{P}}F\left(\tilde{P},\theta\right)=P\left(Z\vert Y,\theta\right)$
    * 引理2 $\to F\left(P\left(Z\vert Y,\theta\right),\theta\right)=\log P\left(Y\vert\theta\right)$
    * 引理1+引理2 $\to\arg\max_{\theta}\log P\left(Y\vert\theta\right)=\arg\max_{\theta}\arg\max_{\tilde{P}}F\left(\tilde{P},\theta\right)$
    * $F\left(\tilde{P}^{(i+1)},\theta\right)=Q\left(\theta,\theta^{(i)}\right)+H\left(\tilde{P}^{(i+1)}\right)$,
其中$\tilde{P}^{(i+1)}\left(Z\right)=P\left(Z\vert Y,\theta^{(i)}\right)$
    * 对 $F$ 求极大($\tilde{P}$)-极大($\theta$) 算法等价于 EM 算法


